In [2]:
import sqlite3
import requests
import json
import time
import os
from getpass import getpass
from datetime import datetime

# Secure API Key input with validation
API_KEY = getpass("Enter your GHL API Key: ")
    
# Location ID from GHL 
LOCATION_ID = getpass("Enter your GHL location ID: ")

# Configuration
REQUEST_TIMEOUT = 30  # seconds
DELAY_BETWEEN_REQUESTS = 1  # seconds 
MAX_RETRIES = 3

Enter your GHL API Key:  ········
Enter your GHL location ID:  ········


In [3]:
def connect_to_db(db_path='instagram_leads.db'):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        return conn, cursor
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        raise

conn, cursor = connect_to_db()

# Add necessary columns if they don't exist
columns_to_add = [
    ("sent_to_crm", "INTEGER DEFAULT 0"),
    ("sent_timestamp", "TIMESTAMP"),
    ("retry_count", "INTEGER DEFAULT 0"),
    ("error_message", "TEXT")
]

for column_name, data_type in columns_to_add:
    try:
        cursor.execute(f"ALTER TABLE profiles ADD COLUMN {column_name} {data_type}")
        conn.commit()
        print(f" '{column_name}' column added.")
    except sqlite3.OperationalError as e:
        if "duplicate column name" in str(e):
            print(f" '{column_name}' already exists.")
        else:
            print(f" Error adding column {column_name}: {e}")

print("Database setup complete.")

 'sent_to_crm' column added.
 'sent_timestamp' column added.
 'retry_count' column added.
 'error_message' column added.
Database setup complete.


In [5]:
# Get leads with score >= 20 that haven't been sent to GHL yet
cursor.execute('''
    SELECT 
        username, email, phone, website_link, location, 
        follower_count, lead_score, retry_count
    FROM profiles
    WHERE lead_score >= 20 
    AND (sent_to_crm IS NULL OR sent_to_crm = 0)
    AND (retry_count IS NULL OR retry_count < ?)
    ORDER BY lead_score DESC
''', (MAX_RETRIES,))

leads = cursor.fetchall()
print(f" Found {len(leads)} qualified leads to push to GHL.")

# Optional: Show lead distribution by score
if leads:
    print("\nLead Distribution:")
    cursor.execute('''
        SELECT 
            CASE 
                WHEN lead_score >= 80 THEN 'Excellent (80+)'
                WHEN lead_score >= 60 THEN 'High (60-79)'
                WHEN lead_score >= 40 THEN 'Medium (40-59)'
                ELSE 'Basic (20-39)'
            END as score_range,
            COUNT(*) as count
        FROM profiles
        WHERE username IN ({})
        GROUP BY score_range
        ORDER BY MIN(lead_score) DESC
    '''.format(','.join(['?']*len(leads))), [lead[0] for lead in leads])
    
    for score_range, count in cursor.fetchall():
        print(f"   {score_range}: {count} leads")

 Found 2140 qualified leads to push to GHL.

Lead Distribution:
   Excellent (80+): 8 leads
   High (60-79): 106 leads
   Medium (40-59): 1209 leads
   Basic (20-39): 817 leads


In [6]:
# API request headers
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
    "User-Agent": "InstagramLeadScraper/1.0"
}

# Helper function to format phone numbers consistently
def format_phone(phone_number):
    if not phone_number:
        return ""
    
    # Remove all non-digit characters
    digits_only = ''.join(filter(str.isdigit, phone_number))
    
    # Check if this looks like a valid North American number
    if len(digits_only) == 10:
        return f"+1{digits_only}"
    elif len(digits_only) > 10:
        return f"+{digits_only}"
    else:
        return digits_only  # Return as is if we can't determine format

# Test the API connection before proceeding
try:
    test_response = requests.get(
        "https://rest.gohighlevel.com/v1/locations/",
        headers=headers,
        timeout=REQUEST_TIMEOUT
    )
    if test_response.status_code == 200:
        print("API connection successful")
    else:
        print(f"API connection warning: {test_response.status_code} - {test_response.text}")
except Exception as e:
    print(f"API connection error: {e}")
    print("Will attempt to send leads anyway...")

API connection successful


In [ ]:
import re

# Statistics counters
success_count = 0
fail_count = 0
skip_count = 0

# Timestamp for this batch
batch_timestamp = datetime.now().isoformat()

# Display estimated time
total_leads = len(leads)
est_minutes = total_leads / 60  # Roughly 1 second per lead
print(f"Starting processing of {total_leads} leads (estimated time: ~{est_minutes:.1f} minutes)")

# Helper function to create a valid email
def sanitize_email(username):
    # Remove invalid email characters and spaces
    sanitized = re.sub(r'[^\w.-]', '', username)
    # Ensure no consecutive dots
    sanitized = re.sub(r'\.+', '.', sanitized)
    # Ensure doesn't start or end with dot
    sanitized = sanitized.strip('.')
    
    if not sanitized or not re.match(r'^[\w.-]+$', sanitized):
        # If completely invalid, use a generic email
        return "noemail@example.com"
    return f"{sanitized}@noemail.instagram.com"

# Improved phone format function
def format_phone(phone_number):
    if not phone_number:
        return ""
    
    # Remove all non-digit characters
    digits_only = ''.join(filter(str.isdigit, phone_number))
    
    # Handle empty or too short
    if not digits_only or len(digits_only) < 7:
        return ""
    
    # North American format (most common in Canada)
    if len(digits_only) == 10:
        return f"+1{digits_only}"
    # With country code already
    elif len(digits_only) > 10:
        return f"+{digits_only}"
    # Other formats - just return digits
    else:
        return digits_only

# Process each lead
for lead_index, lead in enumerate(leads):
    username, email, phone, website, location, follower_count, lead_score, retry_count = lead
    retry_count = retry_count or 0
    
    # Display progress every 200 leads
    if lead_index % 200 == 0 and lead_index > 0:
        progress_pct = (lead_index / total_leads) * 100
        print(f"Progress: {lead_index}/{total_leads} leads processed ({progress_pct:.1f}%)")
    
    
    # Prepare contact data with improved email and phone handling
    contact_data = {
        "locationId": LOCATION_ID,
        "firstName": username,
        # Properly sanitize email address
        "email": email if email else sanitize_email(username),
        # Only include phone if it's valid
        "phone": format_phone(phone) if phone else "",
        "customField": [
            {"id": "ig_handle", "value": f"https://instagram.com/{username}"},
            {"id": "location", "value": location if location else ""},
            {"id": "website_link", "value": website if website else ""},
            {"id": "has_email", "value": "Yes" if email else "No"},
            {"id": "follower_count", "value": str(follower_count) if follower_count else "0"},
            {"id": "lead_score", "value": str(lead_score) if lead_score else "0"}
        ],
        "tags": ["IG Lead", "Lash Tech"] + 
                (["High Value"] if lead_score >= 60 else []) +
                (["Has Email"] if email else ["No Email"]) +
                (["Has Phone"] if phone else []),
        "source": "Instagram Scraper"
    }

    # Remove phone completely if it caused an error before
    if retry_count > 0 and "phone number" in (error_message or ""):
        contact_data.pop("phone", None)

    # Send to GHL 
    max_attempts = 2
    for attempt in range(max_attempts):
        try:
            response = requests.post(
                "https://rest.gohighlevel.com/v1/contacts/",
                headers=headers,
                data=json.dumps(contact_data),
                timeout=REQUEST_TIMEOUT
            )
            
            if response.status_code == 200:
                print(f" Sent: @{username} (Score: {lead_score})")
                cursor.execute(
                    "UPDATE profiles SET sent_to_crm = 1, sent_timestamp = ?, retry_count = ?, error_message = NULL WHERE username = ?",
                    (batch_timestamp, retry_count, username)
                )
                success_count += 1
                break
            else:
                error_msg = f"HTTP {response.status_code}: {response.text[:100]}"
                print(f" Failed to send {username}: {error_msg}")
                
                # Handle specific error types
                if "phone number" in error_msg and "phone" in contact_data:
                    # Remove phone from payload on phone number errors
                    contact_data.pop("phone", None)
                    print(f" Removed phone number and retrying...")
                    continue
                elif "email" in error_msg and email is None:
                    # Use a completely generic email on email errors
                    contact_data["email"] = "noemail@example.com"
                    print(f" Using generic email and retrying...")
                    continue
                elif response.status_code == 429:  # Rate limit
                    print(f" Rate limited. Waiting 10 seconds...")
                    time.sleep(10)
                elif attempt < max_attempts - 1:  # Not last attempt
                    print(f" Retrying...")
                    time.sleep(2)
                else:  # Last attempt failed
                    cursor.execute(
                        "UPDATE profiles SET retry_count = ?, error_message = ? WHERE username = ?",
                        (retry_count + 1, error_msg, username)
                    )
                    fail_count += 1
        
        except Exception as e:
            print(f" Exception sending {username}: {str(e)[:100]}")
            if attempt < max_attempts - 1:
                print(f" Retrying...")
                time.sleep(2)
            else:
                cursor.execute(
                    "UPDATE profiles SET retry_count = ?, error_message = ? WHERE username = ?",
                    (retry_count + 1, str(e)[:200], username)
                )
                fail_count += 1
                break
    
    # Commit after each lead to save progress
    conn.commit()
    
    # Small delay between requests to avoid overwhelming the API
    time.sleep(DELAY_BETWEEN_REQUESTS)
  
    

# Final summary
print("\n==== Summary ====")
print(f"Total leads processed: {len(leads)}")
print(f"Successfully sent: {success_count}")
print(f"Failed: {fail_count}")
print(f"Skipped: {skip_count}")

# Close the database connection
conn.close()
print("\nProcess complete. Database connection closed.")

Starting processing of 2140 leads (estimated time: ~35.7 minutes)
 Sent: @venusbeautyca (Score: 90)
 Sent: @all_buffed_up (Score: 90)
 Failed to send volumelash.wholesale.factory: HTTP 400: {"statusCode":400,"message":"The string supplied did not seem to be a phone number","traceId":"f6c6a
 Removed phone number and retrying...
 Sent: @volumelash.wholesale.factory (Score: 80)
 Sent: @lovebeauty_sylviawang (Score: 80)
 Sent: @eli.beauty.lashes (Score: 80)
 Sent: @trancy_tran1987 (Score: 80)
 Sent: @starring_lash_brow (Score: 80)
 Sent: @prepbeautyparlour (Score: 80)
 Sent: @lashes.wholesale.vendor (Score: 70)
 Sent: @beauty_by_armina (Score: 70)
 Sent: @glamourqueenartistry (Score: 70)
 Sent: @tokyobeauteyyc (Score: 70)
 Sent: @lash_manufacturer_cora (Score: 70)
 Sent: @maylashandnailsyyc (Score: 70)
 Failed to send volumelash.wholesale.vendors: HTTP 400: {"statusCode":400,"message":"The string supplied did not seem to be a phone number","traceId":"f3deb
 Removed phone number and retryin

In [ ]:
import csv
from datetime import datetime

# Connect to database
conn = sqlite3.connect('instagram_leads.db')
cursor = conn.cursor()

# Generate timestamp for filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"low_scoring_leads_{timestamp}.csv"

try:
    # Query leads with scores under 20
    cursor.execute('''
        SELECT 
            username, email, phone, website_link, follower_count, 
            location, lead_score, date_scraped 
        FROM profiles
        WHERE lead_score < 20
        ORDER BY lead_score DESC, follower_count DESC
    ''')

    leads = cursor.fetchall()
    print(f"Found {len(leads)} low-scoring leads to export.")

    if leads:
        # Write to CSV
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            # Write header
            writer.writerow([
                'Username', 'Email', 'Phone', 'Website', 'Follower Count', 
                'Location', 'Lead Score', 'Date Scraped'
            ])
            
            # Write data
            for lead in leads:
                writer.writerow(lead)

        print(f"Successfully exported {len(leads)} leads to {filename}")
    
    # Optional: Also export high-scoring leads that failed to send to GHL
    cursor.execute('''
        SELECT 
            username, email, phone, website_link, follower_count, 
            location, lead_score, date_scraped, error_message 
        FROM profiles
        WHERE lead_score >= 20
        AND (sent_to_crm = 0 OR sent_to_crm IS NULL)
        AND retry_count >= ?
        ORDER BY lead_score DESC
    ''', (MAX_RETRIES,))

    failed_leads = cursor.fetchall()
    
    if failed_leads:
        failed_filename = f"failed_ghl_leads_{timestamp}.csv"
        with open(failed_filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            # Write header
            writer.writerow([
                'Username', 'Email', 'Phone', 'Website', 'Follower Count', 
                'Location', 'Lead Score', 'Date Scraped', 'Error Message'
            ])
            
            # Write data
            for lead in failed_leads:
                writer.writerow(lead)
                
        print(f"Also exported {len(failed_leads)} failed high-scoring leads to {failed_filename}")
    
finally:
    conn.close()
    print("Database connection closed.")